In [1]:
import json

from pprint import pprint

dataset_in = "dataset_clicked_2.jsonl"
dataset_out = "dataset1.json"
train_data_out = "train.json"
test_data_out = "test.json"
how_many_clicked = 1000
all_data = []
train_data = []
test_data = []

without_labels_n = 1

# def labels_size(doc):
#     return len(doc[1]["entities"])

n = 0
def get_row_data(line):
    global n
    data = json.loads(line)

    if len(data["text"]) < 6:
        return

    if len(data["labels"]) == 0 and n % 5 != 0:
        return

    n += 1
    return (
        data["text"],
        {"entities": data["labels"]}
    )

with open(dataset_in, "r") as f:
    lines = f.readlines()[:how_many_clicked]
    
    for line in lines:
        if (data := get_row_data(line)) is not None:
            all_data.append(data)

train_data = all_data[:int(len(all_data)*0.8)]
test_data = all_data[:int(len(all_data)*0.2)]

print("all data len", len(all_data))
# print("train data len", len(train_data))
# print("test data len", len(test_data))

n = 0
for e in all_data:
    if len(e[1]["entities"]) == 0:
        n += 1

print("empty labels", n)

all data len 544
empty labels 50


In [1]:
import spacy
import random
import json

nlp = spacy.load("offers.model/")


In [ ]:
nlp.remove_pipe("ner")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)

ner.add_label("Model urządzenia")
ner.add_label("RAM / VRAM")
ner.add_label("Pamięć")
ner.add_label("Przekątna ekranu")
ner.add_label("Kolor")

print("Training started")
nlp.begin_training()

for itn in range(100):
    random.shuffle(all_data)
    losses = {}

    for batch in spacy.util.minibatch(all_data, size=spacy.util.compounding(4.0, 32.0, 1.001)):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]# Update the model
        nlp.update(texts, annotations, losses=losses, drop=0.3)
    
    print(losses)
print("Training finished")

In [ ]:
nlp.to_disk("offers.model")
print("Model saved.")

In [ ]:
# {"text": "President Obama", "labels": [ [10, 15, "PERSON"] ]}

def get_labels(doc):   
    labels = []
    for ent in doc.ents:
        labels.append([ent.start_char, ent.end_char, ent.label_])
        
    return labels

In [ ]:
def create_dataset_row(doc):
    return {"text": doc.text, "labels": get_labels(doc)}

In [ ]:
# Create dataset
# Generate descriptions
import json

offers_in = "offers.jsonl"
offers_out = "offers_dataset2.jsonl"

with open(offers_in) as f_in:
    with open(offers_out, "w") as f_out:
        for line in f_in.readlines()[10000:15000]:
            data = json.loads(line)
            doc = nlp(data['description'])
            row = json.dumps(create_dataset_row(doc))

            f_out.write(f"{row}\n")

In [ ]:
import re

from spacy.lang.pl import Polish
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex

infixes = ["/",] + nlp.Defaults.infixes
nlp.tokenizer.infix_finditer = compile_infix_regex(infixes).finditer

In [3]:
txt = "Sprzedam Xiaomi Redmi Note 7 3/32 Gb w dobrym stanie zainteresowanych osób proszę o kontakt telefoniczny"
doc = nlp(txt)

doc.ents

Model urządzenia
RAM / VRAM
Pamięć


In [3]:
from spacy import displacy
displacy.render(doc, style="ent")